In [1]:
import pandas as pd
import pennylane as qml
import numpy as np

### Lectura del fichero CSV

In [18]:
data = pd.read_csv('Datos_Montabilidad_Anom_447-6-1.csv',delimiter=';')

train_data = data[data['Set'] == 'Train']
test_data = data[data['Set'] == 'Test']

df_binario = train_data[2:].map(lambda x: 1 if x > 0 else 0)
N_1 = len(data.columns[:20])

TypeError: '>' not supported between instances of 'str' and 'int'

In [11]:
N=3
wires = range(N_1)
dev = qml.device("default.qubit", wires)

@qml.qnode(dev)
def circuit(vector):
    qml.BasisEmbedding(features=vector, wires=range(N_1))
    return qml.state()

In [13]:
X_1 = data.iloc[0][:20].to_list()
X = [1,1,1]

In [14]:
print(qml.draw(circuit, expansion_strategy="device")(X_1))

ValueError: Basis state must only consist of 0s and 1s; got ['Train', '0.0', '0.0', '0.0', '0.0', '0.0', '0.127017048426501', '0.0', '0.0154537272455263', '0.0006986299887289884', '0.0116213020951224', '0.0', '0.0', '0.0', '0.0', '0.0', '0.1261003053151767', '0.0074785604566627', '0.0', '0.0']